In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
from langchain_openai import AzureChatOpenAI
import os
import time
import sys


# config.py가 있는 app 폴더까지 경로 추가
sys.path.append(os.path.abspath(".."))  # ../app 까지 이동
from config import settings

# settings에서 환경변수를 가져옴 (config.py에서 이미 로드됨)

class DocumentAnalyzer:
    def __init__(self):
        # Azure AI Search 기본 설정
        self.search_service_name = settings.azure_ai_search_service_name
        self.search_endpoint = f"https://{self.search_service_name}.search.windows.net"
        self.search_credential = AzureKeyCredential(settings.azure_ai_search_api_key)
        
        # 동적으로 인덱스 이름 찾기
        self.index_name = self._get_active_index_name()
        
        # Azure AI Search 클라이언트 설정
        self.search_client = SearchClient(
            endpoint=self.search_endpoint,
            index_name=self.index_name,
            credential=self.search_credential
        )
        
        # Azure Blob Storage 클라이언트 설정
        if settings.azure_storage_account_name and settings.azure_storage_account_key:
            self.blob_service_client = BlobServiceClient(
                account_url=f"https://{settings.azure_storage_account_name}.blob.core.windows.net",
                credential=settings.azure_storage_account_key
            )
        else:
            print("⚠️ Azure Storage 환경변수가 설정되지 않았습니다. 파일 업로드 기능을 사용할 수 없습니다.")
            self.blob_service_client = None
        
        self.container_name = settings.azure_storage_container_name
        
        # Azure OpenAI LLM 설정
        self.llm = AzureChatOpenAI(
            model=settings.azure_openai_deployment_name,
            temperature=0,
            api_version=settings.azure_openai_api_version,
            azure_endpoint=settings.azure_openai_endpoint,
            azure_deployment=settings.azure_openai_deployment_name,
        )
    
    def _get_active_index_name(self) -> str:
        """
        동적으로 활성 인덱스 이름 찾기
        'rag-'로 시작하는 인덱스 중 가장 최신 것을 반환
        """
        try:
            # SearchIndexClient로 인덱스 목록 조회
            index_client = SearchIndexClient(
                endpoint=self.search_endpoint,
                credential=self.search_credential
            )
            
            # 모든 인덱스 조회
            indexes = list(index_client.list_indexes())
            
            # 'rag-'로 시작하는 인덱스들 필터링
            rag_indexes = [idx.name for idx in indexes if idx.name.startswith('rag-')]
            
            if rag_indexes:
                # 가장 최신 인덱스 반환 (이름 기준 정렬)
                latest_index = sorted(rag_indexes, reverse=True)[0]
                print(f"✅ 자동 발견된 인덱스: {latest_index}")
                return latest_index
            else:
                # 기본 인덱스 이름 반환
                fallback_index = "rag-1752025961760"
                print(f"⚠️ rag- 인덱스를 찾을 수 없어서 기본값 사용: {fallback_index}")
                return fallback_index
                
        except Exception as e:
            # 오류 시 기본 인덱스 이름 사용
            fallback_index = "rag-1752025961760"
            print(f"❌ 인덱스 조회 오류, 기본값 사용: {fallback_index} (오류: {str(e)})")
            return fallback_index
    
    def upload_file_to_storage(self, file_content: bytes, filename: str) -> dict:
        """파일을 Azure Blob Storage에 업로드"""
        try:
            if self.blob_service_client is None:
                return {
                    "status": "error",
                    "message": "Azure Storage가 설정되지 않았습니다."
                }
            
            blob_client = self.blob_service_client.get_blob_client(
                container=self.container_name,
                blob=filename
            )
            
            # 파일 업로드
            blob_client.upload_blob(file_content, overwrite=True)
            
            return {
                "status": "success",
                "message": f"파일 '{filename}'이 성공적으로 업로드되었습니다.",
                "filename": filename
            }
        except Exception as e:
            return {
                "status": "error",
                "message": f"파일 업로드 중 오류 발생: {str(e)}"
            }
    
    def upload_resume(self, file_content: bytes, filename: str) -> dict:
        """이력서 파일 업로드"""
        # 이력서 파일명 앞에 prefix 추가
        resume_filename = f"resume_{filename}"
        return self.upload_file_to_storage(file_content, resume_filename)
    
    def upload_job_posting(self, file_content: bytes, filename: str) -> dict:
        """채용공고 파일 업로드"""
        # 채용공고 파일명 앞에 prefix 추가
        job_filename = f"job_{filename}"
        return self.upload_file_to_storage(file_content, job_filename)
    
    def read_resume_file(self, filename: str) -> str:
        """이력서 파일 읽기 (AI Search에서)"""
        try:
            # resume_ prefix가 없으면 추가
            if not filename.startswith("resume_"):
                filename = f"resume_{filename}"
                
            results = self.search_client.search(
                search_text=f"metadata_storage_name:{filename}",
                top=1,
                select=["content", "chunk"]
            )
            
            for result in results:
                content = result.get("content", "") or result.get("chunk", "")
                if content:
                    return content
            
            return f"이력서 파일 '{filename}'을 찾을 수 없습니다. (AI Search 인덱싱 대기 중일 수 있습니다)"
        except Exception as e:
            return f"이력서 파일 읽기 오류: {str(e)}"
    
    def read_job_posting_file(self, filename: str) -> str:
        """채용공고 파일 읽기 (AI Search에서)"""
        try:
            # job_ prefix가 없으면 추가
            if not filename.startswith("job_"):
                filename = f"job_{filename}"
                
            results = self.search_client.search(
                search_text=f"metadata_storage_name:{filename}",
                top=1,
                select=["content", "chunk"]
            )
            
            for result in results:
                content = result.get("content", "") or result.get("chunk", "")
                if content:
                    return content
            
            return f"채용공고 파일 '{filename}'을 찾을 수 없습니다. (AI Search 인덱싱 대기 중일 수 있습니다)"
        except Exception as e:
            return f"채용공고 파일 읽기 오류: {str(e)}"
    
    def wait_for_indexing(self, filename: str, max_wait_time: int = 30) -> bool:
        """AI Search 인덱싱 완료 대기"""
        for _ in range(max_wait_time):
            try:
                results = self.search_client.search(
                    search_text=f"metadata_storage_name:{filename}",
                    top=1,
                    select=["content", "chunk"]
                )
                
                for result in results:
                    content = result.get("content", "") or result.get("chunk", "")
                    if content:
                        return True
                
                time.sleep(1)  # 1초 대기
            except:
                time.sleep(1)
        
        return False
    
    def analyze_match(self, resume_content: str, job_content: str) -> dict:
        """이력서-채용공고 매칭 분석"""
        try:
            prompt = f"""
당신은 전문 채용 컨설턴트입니다. 다음 채용공고와 지원자 이력서를 분석하여 매칭도를 평가해주세요.

**채용공고:**
{job_content}

**지원자 이력서:**
{resume_content}

아래 형식으로 정리해주세요:

## 📊 종합 평가
- 전반적 적합도: XX/100점 (한줄 요약)
- 기술 스택 매칭: XX/100점 (핵심 매칭/부족 기술)
- 경력 충족도: XX/100점 (요구사항 충족 여부)

## ✅ 주요 강점
1. [구체적 강점 1]
2. [구체적 강점 2]
3. [구체적 강점 3]

## ⚠️ 우려 사항
1. [구체적 우려점 1]
2. [구체적 우려점 2]
3. [구체적 우려점 3]

## 💬 추천 면접 질문 5가지
1. [기술 관련 질문]
2. [프로젝트 경험 질문]
3. [문제 해결 질문]
4. [협업/소통 질문]
5. [성장 가능성 질문]
"""
            
            result = self.llm.invoke(prompt)
            
            return {
                "status": "success",
                "analysis": result.content
            }
            
        except Exception as e:
            return {
                "status": "error",
                "message": f"분석 중 오류 발생: {str(e)}"
            }

# 전역 인스턴스 생성
document_analyzer = DocumentAnalyzer()

# 파일 업로드 함수들
def upload_resume_file(file_content: bytes, filename: str) -> dict:
    """이력서 파일 업로드"""
    return document_analyzer.upload_resume(file_content, filename)

def upload_job_posting_file(file_content: bytes, filename: str) -> dict:
    """채용공고 파일 업로드"""
    return document_analyzer.upload_job_posting(file_content, filename)

# 파일 읽기 함수들
def read_resume(filename: str) -> str:
    """이력서 파일 읽기"""
    return document_analyzer.read_resume_file(filename)

def read_job_posting(filename: str) -> str:
    """채용공고 파일 읽기"""
    return document_analyzer.read_job_posting_file(filename)

# 종합 분석 함수
def analyze_candidate_match(resume_file: str, job_file: str) -> dict:
    """이력서-채용공고 종합 분석"""
    resume_content = read_resume(resume_file)
    job_content = read_job_posting(job_file)
    return document_analyzer.analyze_match(resume_content, job_content)

# 인덱싱 대기 함수
def wait_for_file_indexing(filename: str, max_wait_time: int = 30) -> bool:
    """AI Search 인덱싱 완료 대기"""
    return document_analyzer.wait_for_indexing(filename, max_wait_time) 

❌ 인덱스 조회 오류, 기본값 사용: rag-1751935906958 (오류: () The given API key doesn't match service's internal, primary or secondary keys.
Code: 
Message: The given API key doesn't match service's internal, primary or secondary keys.)


In [2]:
# 테스트용 가데이터
resume_sample = """
김개발 이력서

【 기본정보 】
- 이름: 김개발
- 경력: 3년 6개월
- 전공: 컴퓨터공학과

【 기술스택 】
- 프론트엔드: React, TypeScript, Next.js
- 백엔드: Node.js, Express, Python, FastAPI
- 데이터베이스: PostgreSQL, MongoDB
- 클라우드: AWS, Azure
- 기타: Docker, Git, RESTful API

【 프로젝트 경험 】
1. 전자상거래 웹사이트 개발 (2023.03 - 2023.08)
   - React + Node.js 풀스택 개발
   - 결제 시스템 구축 및 관리자 페이지 개발
   - 월 1만명 이용자 처리

2. 채팅 애플리케이션 개발 (2022.09 - 2023.02)
   - WebSocket 실시간 채팅 구현
   - Redis 캐싱 및 세션 관리
   - 동시 접속자 500명 처리

【 업무 경험 】
- (주)테크스타트업 프론트엔드 개발자 (2021.06 - 현재)
- 사용자 인터페이스 설계 및 개발
- API 연동 및 성능 최적화
"""

job_sample = """
백엔드 개발자 채용공고

【 채용정보 】
- 직무: 백엔드 개발자
- 경력: 3년 이상
- 고용형태: 정규직
- 연봉: 4,000-6,000만원

【 필수 요구사항 】
- Python 또는 Java 기반 백엔드 개발 경험 3년 이상
- RESTful API 설계 및 개발 경험
- 데이터베이스 설계 및 쿼리 최적화 경험
- 클라우드 환경(AWS, Azure, GCP) 개발 경험
- Git을 활용한 협업 경험

【 우대사항 】
- FastAPI, Spring Boot 프레임워크 경험
- Docker, Kubernetes 컨테이너 기술 경험
- 대용량 트래픽 처리 경험
- 마이크로서비스 아키텍처 경험
- 팀 리딩 경험

【 주요업무 】
- 서버 사이드 API 개발 및 유지보수
- 데이터베이스 설계 및 성능 최적화
- 클라우드 인프라 구축 및 관리
- 코드 리뷰 및 기술 문서 작성
- 신규 기술 도입 및 아키텍처 개선
"""

print("가데이터 준비 완료!")

가데이터 준비 완료!


In [3]:
# Azure 서비스 없이 직접 텍스트로 분석
try:
    # document_analyzer 인스턴스에서 analyze_match 직접 호출
    result = document_analyzer.analyze_match(resume_sample, job_sample)
    
    print("=== 분석 결과 ===")
    print(f"Status: {result['status']}")
    if result['status'] == 'success':
        print("\n" + result['analysis'])
    else:
        print(f"Error: {result['message']}")
        
except Exception as e:
    print(f"테스트 중 오류: {str(e)}")

=== 분석 결과 ===
Status: error
Error: 분석 중 오류 발생: Error code: 401 - {'error': {'code': '401', 'message': 'Access denied due to invalid subscription key or wrong API endpoint. Make sure to provide a valid key for an active subscription and use a correct regional API endpoint for your resource.'}}


In [4]:
# Azure 연결 상태 확인
print("환경변수 체크:")
print(f"AZURE_OPENAI_ENDPOINT: {os.getenv('AZURE_OPENAI_ENDPOINT', 'NOT SET')}")
print(f"AZURE_OPENAI_DEPLOYMENT_NAME: {os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME', 'NOT SET')}")
print(f"AZURE_AI_SEARCH_SERVICE_NAME: {os.getenv('AZURE_AI_SEARCH_SERVICE_NAME', 'NOT SET')}")

환경변수 체크:
AZURE_OPENAI_ENDPOINT: https://user04-openai-001.openai.azure.com/
AZURE_OPENAI_DEPLOYMENT_NAME: dev-gpt-4o
AZURE_AI_SEARCH_SERVICE_NAME: user04-ai-search
